In [6]:
import cv2
from inference_sdk import InferenceHTTPClient
from inference.core.utils.image_utils import load_image
from inference import InferencePipeline
from inference.core.interfaces.camera.entities import VideoFrame
# import supervision to help visualize our predictions
import supervision as sv
from inference_sdk import InferenceHTTPClient
from PIL import Image
import base64
import io
import os
import re


In [2]:
client = InferenceHTTPClient(
    api_url=f"http://127.0.0.1:9001",
    api_key="HHOQQa2OcXtsGddANg0W",
)


In [ ]:

# to list active pipelines
client.list_inference_pipelines()

In [ ]:
while True:
    result = client.consume_inference_pipeline_result(pipeline_id="<PIPELINE-ID>")
    if not result["outputs"] or not result["outputs"][0]:
        # "outputs" key contains list of workflow results - why list? InferencePipeline can 
        # run on multiple video sources at the same time - each "round" it attempts to 
        # grab single frame from all sources and run through Workflows Execution Engine
        # * when sources are inactive that may not be possible, hence empty list can be returned
        # * when one of the source do not provide frame (for instance due to batch collection timeout)
        # outputs list may contain `None` values!
        continue
    # let's assume single source
    source_result = result["outputs"][0]
    image, _ = load_image(source_result["preview"])  # "preview" is the name of workflow output with image
    cv2.imshow("frame", image)
    cv2.waitKey(1)

In [7]:
def my_sink(result: dict, video_frame: VideoFrame):
    print(result) # here you can find dictionary with outputs from your workflow

In [8]:
count=1
predict={}
frame=0
def my_custom_sink(predictions: dict, video_frame: VideoFrame):
    #print(predictions['label_visualization'],video_frame.image) # do something with the predictions of each frame
    detections = sv.Detections.from_inference(predictions["label_visualization"])
    print(detections)
    #cv2.imwrite('prediction/frame1.jpg',video_frame.image)
    cv2.imshow(predictions["label_visualization"], "image")
    cv2.waitKey(1)

In [10]:

# initialize a pipeline object
pipeline = InferencePipeline.init_with_workflow(
    api_key="HHOQQa2OcXtsGddANg0W",
    workspace_name="robot-application",
    workflow_id="custom-workflow-q1t",
    video_reference="material/mpv-shot0003.jpg" , # Path to video, device id (int, usually 0 for built in webcams), or RTSP stream url
    max_fps=1,
    on_prediction=my_sink,
    image_input_name="image",
)
pipeline.start() #start the pipeline
pipeline.join() #wait for the pipeline thread to finish

{'label_visualization': <inference.core.workflows.execution_engine.entities.base.WorkflowImageData object at 0x000001E32A075150>}


In [15]:
result = client.run_workflow(
    workspace_name="robot-application",
    workflow_id="custom-workflow-q1t",
    images={
        "image": ["material/mpv-shot0003.jpg"]
    },

)

In [16]:
result

[{'label_visualization': '...'}]

In [18]:
prediction_folder="prediction/"

target=prediction_folder+"prediction.jpg"
imgdata = base64.b64decode(result[0]["label_visualization"])
# I assume you have a way of picking unique filenames
with open(target, 'wb') as f:
    f.write(imgdata)
image = cv2.imread(target)
# cv2.imshow(head,image)
# cv2.waitKey(0) #close windows only after any key press
#cv2.destroyAllWindows()
        